<a href="https://colab.research.google.com/github/Nikhil4002-50-82/Churn-Prediction/blob/main/ChurnPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded=files.upload()

!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
!kaggle datasets download blastchar/telco-customer-churn

Dataset URL: https://www.kaggle.com/datasets/blastchar/telco-customer-churn
License(s): copyright-authors
  0% 0.00/172k [00:00<?, ?B/s]
100% 172k/172k [00:00<00:00, 446MB/s]


In [3]:
!unzip telco-customer-churn.zip

Archive:  telco-customer-churn.zip
  inflating: WA_Fn-UseC_-Telco-Customer-Churn.csv  


In [5]:
!pip install ydata-profiling
import pandas as pd
from ydata_profiling import ProfileReport

data=pd.read_csv("/content/WA_Fn-UseC_-Telco-Customer-Churn.csv")
profile = ProfileReport(data,title="Profiling Report")
profile.to_file("report1.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 21/21 [00:01<00:00, 15.26it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
y=data["Churn"]
X=data.drop(columns=["customerID","Churn"])

In [7]:
print(X.info())
print(X.isna().sum())
print(f"shape of the data : {X.shape}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   object 
 16  PaymentMethod     7043 non-null   object 


In [8]:
y.value_counts()

,count
Churn,
No,5174
Yes,1869


In [9]:
y.sample(5)

,Churn
6598,No
4434,No
4267,No
3571,No
5915,No


In [11]:
X.sample()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
6280,Female,0,No,Yes,23,Yes,No,DSL,No,No,No,Yes,No,No,One year,Yes,Electronic check,49.45,1119.35


In [12]:
import numpy as np
X["TotalCharges"]=X["TotalCharges"].replace(" ",np.nan)
X["TotalCharges"]=X["TotalCharges"].astype(float)

In [13]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   object 
 16  PaymentMethod     7043 non-null   object 


In [14]:
X.isna().sum()

,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0
OnlineBackup,0


In [28]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [34]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_test=le.transform(y_test)

In [57]:
from sklearn import set_config
set_config(display="diagram")
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler

numericFeatures=[4,17,18]
categoricalFeatures=[0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16]

# for i in numericFeatures:
#     print(i, X_train.columns[i], X_train.dtypes[i])
# print()
# for i in categoricalFeatures:
#     print(i, X_train.columns[i], X_train.dtypes[i])

categoricalPipeline=Pipeline([
    ("encoder",OneHotEncoder(drop="first",sparse_output=False))
])
numericalPipeline=Pipeline([
    ("imputer",SimpleImputer(strategy="mean")),
    ("scaler",StandardScaler())
])

transformer=ColumnTransformer([
    ("categorical",categoricalPipeline,categoricalFeatures),
    ("numerical",numericalPipeline,numericFeatures)
])

pipeline=Pipeline([
    ("preprocessor",transformer)
])

X_train=pipeline.fit_transform(X_train)
X_test=pipeline.transform(X_test)

featureNames=pipeline.named_steps["preprocessor"].get_feature_names_out()
X_train=pd.DataFrame(X_train,columns=featureNames)
X_test=pd.DataFrame(X_test,columns=featureNames)
y_train=pd.DataFrame(y_train,columns=["Churn"])
y_test=pd.DataFrame(y_test,columns=["Churn"])

pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse_output=False))]),
                                                  [0, 1, 2, 3, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15,
                                                   16]),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  [4, 17, 18])]))])

In [58]:
X_train.head()

,categorical__categorical__categorical__x0_1.0_1.0_1.0,categorical__categorical__categorical__x1_1.0_1.0_1.0,categorical__categorical__categorical__x2_1.0_1.0_1.0,categorical__categorical__categorical__x3_1.0_1.0_1.0,categorical__categorical__categorical__x7_1.0_1.0_1.0,categorical__categorical__categorical__x8_1.0_1.0_1.0,categorical__categorical__categorical__x9_1.0_1.0_1.0,categorical__categorical__categorical__x10_1.0_1.0_1.0,categorical__categorical__categorical__x11_1.0_1.0_1.0,categorical__categorical__categorical__x12_1.0_1.0_1.0,categorical__categorical__categorical__x13_1.0_1.0_1.0,categorical__categorical__categorical__x14_1.0_1.0_1.0,categorical__categorical__categorical__x15_1.0_1.0_1.0,categorical__categorical__categorical__x16_1.1329009519499638_1.0_1.0,categorical__categorical__numerical__x4_1.8914360090724638_1.0,categorical__numerical__categorical__x5_1.0_1.8914360090725342,numerical__categorical__categorical__x6_1.0_1.0,numerical__numerical__numerical__x17,numerical__numerical__numerical__x18
0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,-0.634162,-0.528699,1.259591
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,-0.634162,-0.528699,1.259591
2,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,-0.634162,1.891436,-0.793909
3,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,-0.634162,-0.528699,1.259591
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.576884,-0.528699,-0.793909


In [60]:
y_train.head()

,Churn
0,0
1,0
2,0
3,0
4,0


In [61]:
print(X_train.shape)
print(y_train.shape)

(4930, 19)
(4930, 1)


In [62]:
import tensorflow as tf
from tensorflow import keras

model=keras.Sequential([
    keras.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(64,activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(32,activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(16,activation="relu"),
    keras.layers.Dense(1,activation="sigmoid")
])

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [63]:
model.fit(X_train,y_train,epochs=100)

Epoch 1/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7079 - loss: 0.5712
Epoch 2/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7513 - loss: 0.4832
Epoch 3/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7555 - loss: 0.4879
Epoch 4/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7614 - loss: 0.4770
Epoch 5/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7621 - loss: 0.4848
Epoch 6/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7725 - loss: 0.4643
Epoch 7/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7706 - loss: 0.4827
Epoch 8/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7763 - loss: 0.4793
Epoch 9/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7752 - loss: 0.4719
Epoch 10/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7756 - loss: 0.4728
Epoch 11/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7774 - loss: 0.4652
Epoch 12/100
155/155 ━━━━━━━━━━━━━━━━━━━━

In [77]:
report2=ProfileReport(X_train,title="Profiling Report For Training Data After Preprocessing")
report2.to_file("report2.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 19/19 [00:00<00:00, 44.37it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [78]:
pred=pd.DataFrame(model.predict(X_test),columns=["PredictedValue"])
y_test=y_test.rename(columns={"Churn":"ActualValue"})
result=pd.concat([y_test,pred],axis=1)
result.sample(10)

67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


,ActualValue,PredictedValue
934,0,0.459966
1742,1,0.532464
556,0,0.040509
996,1,0.477684
632,1,0.136157
321,0,0.170130
580,0,0.187477
635,0,0.167193
819,0,0.183513
1250,0,0.092877
